*JJ McCauley*

## Task 1: Regression for Computer Hardware Dataset

In [12]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Retreiving the dataset
computer_hardware_df = pd.read_csv("machine.data")
X = computer_hardware_df.iloc[:, 3:8]  # Import columns 3-8
y = computer_hardware_df.iloc[:, -1]

# Determining correlations to select 4 most important attributes
correlations = pd.concat([X, y], axis=1).corr()
target_correlations = correlations.iloc[:-1, -1]
chosen_features = target_correlations.abs().sort_values(ascending=False).head(4)
X_new = X[chosen_features.index]

# Splitting the dataset
x_train, x_test, y_train, y_test = train_test_split(X_new, y, test_size=.4, random_state=42)

# Building the Linear Regression Model 
lr_model = LinearRegression(n_jobs=-1)
lr_model.fit(x_train, y_train)
y_pred = lr_model.predict(x_test)

# Displaying the metrics
print("Mean Absolute Error (MAE): ", metrics.mean_absolute_error(y_test, y_pred))
print("Mean Squared Error (MSE): ", metrics.mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error: ", np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print("r2 Score: ", metrics.r2_score(y_test, y_pred))

Mean Absolute Error (MAE):  34.164566195073476
Mean Squared Error (MSE):  2597.292739775501
Root Mean Squared Error:  50.963641351217255
r2 Score:  0.8553522616522726


## Task 2: Clustering on Hand-Written Digits

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.datasets import load_digits

# Load in the data from sklearn library
digits_loaded_data = load_digits()
X = digits_loaded_data.data
y = digits_loaded_data.target

# Conduct PCA to reduce dimensionality
